In [1]:
import torch, platform
print("Python:", platform.python_version())
print("Torch :", torch.__version__, " CUDA build:", torch.version.cuda)
print("CUDA avail:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


Python: 3.10.14
Torch : 2.3.0  CUDA build: 11.8
CUDA avail: True
GPU: NVIDIA GeForce RTX 4070 Laptop GPU


In [3]:
from pathlib import Path
DATA_DIR = Path(r"Red_reflector_detector.v3-annotatae_3_ver.yolov8")  # <-- 본인 경로
data_yaml = DATA_DIR / "data.yaml"
assert data_yaml.exists(), f"data.yaml 못 찾음: {data_yaml}"
print("Using:", data_yaml)


Using: Red_reflector_detector.v3-annotatae_3_ver.yolov8\data.yaml


In [5]:
from pathlib import Path
def count_pairs(split):
    img = list((DATA_DIR/split/"images").glob("*.*"))
    lbl = list((DATA_DIR/split/"labels").glob("*.txt"))
    return len(img), len(lbl)

for sp in ["train","valid","test"]:
    p = DATA_DIR/sp
    if p.exists():
        ni, nl = count_pairs(sp)
        print(f"{sp:5s}  images={ni:4d}  labels={nl:4d}")


train  images= 491  labels= 491
valid  images= 120  labels= 120


In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8m.pt")   # m 모델
results = model.train(
    data=str(data_yaml),
    imgsz=640,
    epochs=80,
    batch=-1,          # 가능한 최대 배치 자동
    device=0,          # GPU 0번 (없으면 'cpu')
    amp=True,          # 혼합정밀(FP16)
    workers=4,
    cos_lr=True,
    patience=20,       # early stopping
    project="runs/detect", name="train_y8m_ud"
)
print("save_dir:", results.save_dir)
